In [1]:
import pandas as pd
from datatable import dt, f

# Aquisição dos dados brutos

como os dados foram adquiridos (decrição de atualizações na base e etc)


## Áreas STEM

- 51: ciências biológicas e correlatas

- 54: matemática e estatística

- 53: ciências físicas

- 61: computação

- 71: engenharias e correlatas

In [2]:
def filter_dataset(dataset):
  df = dataset[
    (
      (f.cod_area == 51)
      |
      (f.cod_area == 54)
      |
      (f.cod_area == 53)
      |
      (f.cod_area == 61)
      |
      (f.cod_area == 61)
      |
      (f.cod_area == 71)
    ) & (
      (f.ingressantes > 0)
      &
      (f.concluintes > 0)
    )
    , :]
  return df

In [3]:
def get_path_table_by_year(year):
  path_table = '/'.join([
    '../data/raw',
    'censo_{year}.zip',
    'Microdados do Censo da Educaç╞o Superior {year}',
    'dados',
    'MICRODADOS_CADASTRO_CURSOS_{year}.CSV'
  ]).format(year = year)
  return path_table

In [4]:
def read_datasets(from_year, to_year, columns):
  dfs = list()
  for y in range(from_year, to_year + 1):
    path_table = get_path_table_by_year(y)
    dataset = dt.fread(path_table, columns=columns)
    df_filter = filter_dataset(dataset)
    dfs.append(df_filter)
  df =  dt.rbind(*dfs)
  df = df.sort(f.ano, f.cod_municipio, f.area, f.curso, -f.concluintes_fem)
  df = df.to_pandas()
  df['rede_publica'] = df['rede_publica'].map({1: True, 2: False})
  df['presencial'] = df['presencial'].map({1: True, 2: False})
  df['bacharel'] = df['bacharel'].map({1: True, 3: False})
  df['frac_ingressantes'] = df['ingressantes_fem'] / df['ingressantes']
  df['frac_concluintes'] = df['concluintes_fem'] / df['concluintes']
  df.dropna(inplace=True)
  return df

## Colunas

- NU_ANO_CENSO: Ano do Censo

- NO_REGIAO: Nome da região geográfica do local de oferta do curso

- SG_UF: Sigla da Unidade da Federação do local de oferta do curso

- NO_MUNICIPIO: Nome do Município do local de oferta do curso

- CO_MUNICIPIO: Código do IBGE do municipio do local de oferta do curso

- IN_CAPITAL: Informa se o local de oferta do curso está localizado em capital de Estado
	- 0. Não
	- 1. Sim
	- (.) Não aplicável (Curso a distância)

- TP_REDE: Rede de Ensino
	- 1. Pública
	- 2. Privada

- CO_CINE_AREA_ESPECIFICA: Nome da área específica, conforme adaptação da Classificação Internacional Normalizada da Educação Cine/Unesco

- NO_CINE_ROTULO: Nome do curso, conforme adaptação da Classificação Internacional Normalizada da Educação Cine/Unesco

- TP_GRAU_ACADEMICO: Tipo do grau acadêmico conferido ao ao aluno pela conclusão dos requisitos exigidos pelo curso
	- 1. Bacharelado
	- 2. Licenciatura
	- 3. Tecnológico
	- 4. Bacharelado e Licenciatura
	- (.) Não aplicável (cursos com nivel acadêmico igual a sequencial de formação específica ou  cursos de área básica de curso identificada pela variável TP_ATRIBUTO_INGRESSO)

- TP_MODALIDADE_ENSINO: Tipo da modalidade de ensino do curso
	- 1. Presencial
	- 2. Curso a distância

- QT_ING: Quantidade de ingressante no curso

- QT_ING_FEM: Quantidade de ingressante do sexo feminino no curso

- QT_CONC: Quantidade de concluinte no curso

- QT_CONC_FEM: Quantidade de concluinte no curso do sexo feminino

In [5]:
columns = {
  'NU_ANO_CENSO': 'ano', #  Ano do Censo
  'NO_REGIAO': 'regiao', #  Nome da região geográfica do local de oferta do curso
  'SG_UF': 'uf', #  Sigla da Unidade da Federação do local de oferta do curso
  'CO_MUNICIPIO': 'cod_municipio', #  Código do IBGE do municipio do local de oferta do curso
  'NO_MUNICIPIO': 'nome_municipio', #  Nome do Município do local de oferta do curso
  'IN_CAPITAL': 'capital', #  Informa se o local de oferta do curso está localizado em capital de Estado
  'CO_CINE_AREA_ESPECIFICA': 'cod_area', # Código de identificação da área específica, conforme adaptação da Classificação Internacional Normalizada da Educação Cine/Unesco
  'NO_CINE_AREA_ESPECIFICA': 'area', #  Nome da área específica, conforme adaptação da Classificação Internacional Normalizada da Educação Cine/Unesco
  'NO_CINE_ROTULO': 'curso', #  Nome do curso, conforme adaptação da Classificação Internacional Normalizada da Educação Cine/Unesco
  'TP_REDE': 'rede_publica', #  Rede de Ensino
  'TP_MODALIDADE_ENSINO': 'presencial', #  Tipo da modalidade de ensino do curso
  'TP_GRAU_ACADEMICO': 'bacharel', #  Tipo do grau acadêmico conferido ao ao aluno pela conclusão dos requisitos exigidos pelo curso
  'QT_ING': 'ingressantes', #  Quantidade de ingressante no curso
  'QT_ING_FEM': 'ingressantes_fem', #  Quantidade de ingressante do sexo feminino no curso
  'QT_CONC': 'concluintes', #  Quantidade de concluinte no curso
  'QT_CONC_FEM': 'concluintes_fem', #  Quantidade de concluinte no curso do sexo feminino
  ...: None,
}

In [6]:
from_year = 2010
to_year = 2019
df = read_datasets(from_year, to_year, columns)
df = df[list(columns.values())[:-1] + ['frac_ingressantes', 'frac_concluintes']]
df

,ano,regiao,uf,cod_municipio,nome_municipio,capital,cod_area,area,curso,rede_publica,presencial,bacharel,ingressantes,ingressantes_fem,concluintes,concluintes_fem,frac_ingressantes,frac_concluintes
0,2010,Norte,RO,1100023.0,Ariquemes,False,61,Computação e Tecnologias da Informação e Comun...,Sistemas de informação,False,True,True,2,0,15,1,0.000000,0.066667
1,2010,Norte,RO,1100049.0,Cacoal,False,51,Ciências biológicas e correlatas,Biologia,False,True,True,21,13,6,4,0.619048,0.666667
2,2010,Norte,RO,1100049.0,Cacoal,False,61,Computação e Tecnologias da Informação e Comun...,Sistemas de informação,False,True,False,20,4,25,5,0.200000,0.200000
3,2010,Norte,RO,1100049.0,Cacoal,False,71,Engenharia e profissões correlatas,Gestão ambiental,False,True,False,61,26,27,12,0.426230,0.444444
4,2010,Norte,RO,1100049.0,Cacoal,False,71,Engenharia e profissões correlatas,Gestão ambiental,False,False,False,88,48,10,5,0.545455,0.500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48779,2019,Centro-Oeste,DF,5300108.0,Brasília,True,71,Engenharia e profissões correlatas,Gestão ambiental,False,False,False,4,3,2,1,0.750000,0.500000
48780,2019,Centro-Oeste,DF,5300108.0,Brasília,True,71,Engenharia e profissões correlatas,Gestão ambiental,False,False,False,4,0,1,1,0.000000,1.000000
48781,2019,Centro-Oeste,DF,5300108.0,Brasília,True,71,Engenharia e profissões correlatas,Gestão ambiental,False,False,False,28,7,4,0,0.250000,0.000000
48782,2019,Centro-Oeste,DF,5300108.0,Brasília,True,54,Matemática e estatística,Estatística,True,True,True,88,23,23,6,0.261364,0.260870


In [7]:
output_file = '../data/cursos.csv.zip'
df.to_csv(output_file, index=False)

In [8]:
df_read = pd.read_csv(
  output_file, dtype={'cod_municipio': int})
df_read

,ano,regiao,uf,cod_municipio,nome_municipio,capital,cod_area,area,curso,rede_publica,presencial,bacharel,ingressantes,ingressantes_fem,concluintes,concluintes_fem,frac_ingressantes,frac_concluintes
0,2010,Norte,RO,1100023,Ariquemes,False,61,Computação e Tecnologias da Informação e Comun...,Sistemas de informação,False,True,True,2,0,15,1,0.000000,0.066667
1,2010,Norte,RO,1100049,Cacoal,False,51,Ciências biológicas e correlatas,Biologia,False,True,True,21,13,6,4,0.619048,0.666667
2,2010,Norte,RO,1100049,Cacoal,False,61,Computação e Tecnologias da Informação e Comun...,Sistemas de informação,False,True,False,20,4,25,5,0.200000,0.200000
3,2010,Norte,RO,1100049,Cacoal,False,71,Engenharia e profissões correlatas,Gestão ambiental,False,True,False,61,26,27,12,0.426230,0.444444
4,2010,Norte,RO,1100049,Cacoal,False,71,Engenharia e profissões correlatas,Gestão ambiental,False,False,False,88,48,10,5,0.545455,0.500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48711,2019,Centro-Oeste,DF,5300108,Brasília,True,71,Engenharia e profissões correlatas,Gestão ambiental,False,False,False,4,3,2,1,0.750000,0.500000
48712,2019,Centro-Oeste,DF,5300108,Brasília,True,71,Engenharia e profissões correlatas,Gestão ambiental,False,False,False,4,0,1,1,0.000000,1.000000
48713,2019,Centro-Oeste,DF,5300108,Brasília,True,71,Engenharia e profissões correlatas,Gestão ambiental,False,False,False,28,7,4,0,0.250000,0.000000
48714,2019,Centro-Oeste,DF,5300108,Brasília,True,54,Matemática e estatística,Estatística,True,True,True,88,23,23,6,0.261364,0.260870


In [9]:
df_read.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48716 entries, 0 to 48715
Data columns (total 18 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   ano                48716 non-null  int64  
 1   regiao             48716 non-null  object 
 2   uf                 48716 non-null  object 
 3   cod_municipio      48716 non-null  int64  
 4   nome_municipio     48716 non-null  object 
 5   capital            48716 non-null  bool   
 6   cod_area           48716 non-null  int64  
 7   area               48716 non-null  object 
 8   curso              48716 non-null  object 
 9   rede_publica       48716 non-null  bool   
 10  presencial         48716 non-null  bool   
 11  bacharel           48716 non-null  bool   
 12  ingressantes       48716 non-null  int64  
 13  ingressantes_fem   48716 non-null  int64  
 14  concluintes        48716 non-null  int64  
 15  concluintes_fem    48716 non-null  int64  
 16  frac_ingressantes  487